In [16]:
# reference: https://github.com/StefOe/all-conv-pytorch/blob/master/cifar10.ipynb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import sys
sys.path.append('..')
# import utils_for_google_drive

# from allconvnet import AllConvNet
import os
import pdb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class AllConvNet(nn.Module):
    def __init__(self, in_channels, num_classes=10, dropout=True):
        super().__init__()
        self.dropout = dropout
        self.conv1 = nn.Conv2d(in_channels, 96, 3, padding=1)
        self.conv2 = nn.Conv2d(96, 96, 3, padding=1)
        self.conv3 = nn.Conv2d(96, 96, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(96, 192, 3, padding=1)
        self.conv5 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv6 = nn.Conv2d(192, 192, 3, stride=2, padding=1)
        # self.conv7 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv7 = nn.Conv2d(192, 192, 3)
        self.conv8 = nn.Conv2d(192, 192, 1)
        self.conv_class = nn.Conv2d(192, num_classes, 1)

    def forward(self, x):
        if self.dropout:
            x = F.dropout(x, .2)
        x = F.relu(self.conv1(x))
        # print(x)
        x = F.relu(self.conv2(x))
        #print(x)
        x = F.relu(self.conv3(x))
        # print(x)
        # pdb.set_trace()

        if self.dropout:
            x = F.dropout(x, .5)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        if self.dropout:
            x = F.dropout(x, .5)
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv_class(x))
        # print(x)
        x = x.reshape(x.size()[0], x.size()[1], -1).mean(-1)
#         x = F.adaptive_avg_pool2d(x, 1)
#         # print(x)
#         x.squeeze_(-1)
#         # print(x)
#         x.squeeze_(-1)
        # print(x)
        return x

In [3]:
# loading dataset
datapath = 'E:/Datasets/CIFAR10'
# trainset = datasets.CIFAR10(root=datapath,
#                             train=True,
#                             download=False,
#                             # transform=transform  # 按照上面定义的transform格式转换下载的数据
#                             )
# mean = trainset.data.mean(axis=(0,1,2))/255
# std = trainset.data.std(axis=(0,1,2))/255

# train_transform = transforms.Compose([transforms.ToTensor(), # 将PILImage转换为张量 
#                                       # transforms.RandomCrop(32, padding=4),
#                                       # transforms.RandomHorizontalFlip(),
#                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]  # 将[0, 1]归一化到[-1, 1]
#                                       # transforms.Normalize(mean, std)]
#                                      )
# test_transform = transforms.Compose([transforms.ToTensor(),
#                                      transforms.Normalize(mean, std)])

transform = transforms.Compose([transforms.ToTensor(), # 将PILImage转换为张量  
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 将[0, 1]归一化到[-1, 1]

trainset = datasets.CIFAR10(root=datapath,
                            train=True,
                            download=False,
                            # transform=train_transform  # 按照上面定义的transform格式转换下载的数据
                            transform=transform
                            )
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=8,  # 每个batch载入的图片数量，默认为1
                                          shuffle=True,
                                          num_workers=2  # 载入训练数据所需的子任务数
                                          )

testset = datasets.CIFAR10(root=datapath,
                           train=False,
                           download=False,
                           # transform=test_transform
                           transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=16,
                                         shuffle=False,
                                         num_workers=2)

cifar10_classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# mean, std  (array([0.49139968, 0.48215841, 0.44653091]), array([0.24703223, 0.24348513, 0.26158784]))

In [19]:
trainloader.dataset, testloader.dataset, len(testloader.dataset)

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: E:/Datasets/CIFAR10
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: E:/Datasets/CIFAR10
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 10000)

In [13]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
torchvision.utils.save_image(images[1], 'test.jpg')

images.shape, labels, cifar10_classes[labels[1]]

(torch.Size([8, 3, 32, 32]), tensor([3, 5, 5, 3, 7, 4, 1, 9]), 'dog')

In [6]:
# load model
net = AllConvNet(3)
net

AllConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (conv_class): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
)

In [14]:
x = torch.randn(1, 3, 32, 32)
y = net(x)
y, y.shape

(tensor([[0.0000, 0.0205, 0.0317, 0.0299, 0.0000, 0.0607, 0.0712, 0.0000, 0.0000,
          0.0000]], grad_fn=<MeanBackward1>),
 torch.Size([1, 10]))

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 250, 300], gamma=0.1)

In [15]:
# training
def train(epoch):
    net.train()
    train_loss = 0.0
    for batch_idx, data in enumerate(trainloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
#         print(outputs)
#         print(labels)
#         pdb.set_trace()
        loss = criterion(outputs, labels)
        # print(loss)
        # pdb.set_trace()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 500 == 499:  # print loss every 500 batch
            print('Train Epoch: {} - [{}/{} ({:.0f}%)]   loss: {:.6f}'.format(epoch, (batch_idx+1) * len(inputs), len(trainloader.dataset), 
                                                                              100. * batch_idx / len(trainloader), train_loss / 500))
            train_loss = 0.0
      
    print('Saving epoch {} model ...'.format(epoch + 1))
    state = {
        'net': net.state_dict(), 
        'epoch': epoch + 1}
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/allconvnet_cifar10_epoch_%d.ckpt' % (epoch + 1))
            
for epoch in range(1):
    train(epoch)
print('Finished training.')

Train Epoch: 0 - [4000/50000 (8%)]   loss: 2.303235
Train Epoch: 0 - [8000/50000 (16%)]   loss: 2.302552
Train Epoch: 0 - [12000/50000 (24%)]   loss: 2.302932
Train Epoch: 0 - [16000/50000 (32%)]   loss: 2.302716
Train Epoch: 0 - [20000/50000 (40%)]   loss: 2.302590
Train Epoch: 0 - [24000/50000 (48%)]   loss: 2.302585
Train Epoch: 0 - [28000/50000 (56%)]   loss: 2.302585
Train Epoch: 0 - [32000/50000 (64%)]   loss: 2.302585
Train Epoch: 0 - [36000/50000 (72%)]   loss: 2.302585
Train Epoch: 0 - [40000/50000 (80%)]   loss: 2.302585
Train Epoch: 0 - [44000/50000 (88%)]   loss: 2.302585
Train Epoch: 0 - [48000/50000 (96%)]   loss: 2.302585
Saving epoch 1 model ...
Finished training.


In [24]:
def test(epoch, best_loss, best_epoch):
    net.eval()
    test_loss = 0
    correct = 0
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        # sum up batch loss
        test_loss += criterion(outputs, labels).item()
        # get the index of the max log-probability
        _, pred = torch.max(outputs.data, 1)
        # pred = outputs.data.max(1, keepdim=True)[1]
        correct += (pred == labels).cpu().sum().item()
        # correct += (pred.eq(target.data.view_as(pred)).long().cpu().sum())
        
    test_loss /= len(testloader.dataset)  # 10000
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(net, 'best.pt')
    
    return best_loss, best_epoch

In [25]:
best_loss = float('inf')
best_epoch = -1
net = net.to(device)

for epoch in range(1):
    # scheduler.step()
    # train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)


Test set: Average loss: 0.1439, Accuracy: 1004/10000 (10%)

